In [ ]:
!pip install stable-baselines3
!pip install sb3-contrib
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.6 MB/s eta 0:00:00


In [ ]:
import gym
import numpy as np
from stable_baselines3 import PPO, A2C, SAC, TD3, DQN
from sb3_contrib import QRDQN, TQC
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances
from typing import Any, Dict
import torch
import torch.nn as nn
from stable_baselines3.common.callbacks import EvalCallback
import torch as th
import pandas as pd

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
N_TRIALS = 100
N_JOBS = 1
N_STARTUP_TRIALS = 5
N_EVALUATIONS = 2
N_TIMESTEPS = int(2E4)
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_ENVS = 5
N_EVAL_EPISODES = 10
TIMEOUT = int(60 * 15)
ENV_ID = 'CartPole-v1'
DEFAULT_HYPERPARAMS = {
    'policy': 'MlpPolicy',
    'env': ENV_ID
}

In [ ]:
def sample_a2c_params(trial: optuna.Trial) -> Dict[str, Any]:

  gamma = 1.0 - trial.suggest_float('gamma', 0.0001, 0.1, log=True)
  max_grad_norm = trial.suggest_float('max_grad_norm', 0.3, 5.0, log=True)
  n_steps = 2 ** trial.suggest_int('exponent_n_steps', 3, 10)
  learning_rate = trial.suggest_float('learning_rate', 1e-5, 1, log=True)
  net_arch = trial.suggest_categorical('net_arch', ['tiny', 'small'])
  activation_fn = trial.suggest_categorical('activation_fn', ['tanh', 'relu'])

  trial.set_user_attr('gamma', gamma)
  trial.set_user_attr('n_steps', n_steps)

  net_arch = [
      {'pi': [64], 'vf': [64]} if net_arch == 'tiny'
      else {'pi': [64, 64], 'vf': [64, 64]}
  ]

  activation_fn = {'tanh': nn.Tanh, 'relu': nn.ReLU}[activation_fn]

  return {
      'n_steps': n_steps,
      'gamma': gamma,
      'learning_rate': learning_rate,
      'max_grad_norm': max_grad_norm,
      'policy_kwargs': {
          'net_arch': net_arch,
          'activation_fn': activation_fn
      }
  }

In [ ]:
class TrialEvalCallback(EvalCallback):
  def __init__(
      self,
      eval_env: gym.Env,
      trial: optuna.Trial,
      n_eval_episodes: int = 5,
      eval_freq: int = 10000,
      deterministic: bool = True,
      verbose: int = 0
  ):
      super().__init__(
          eval_env=eval_env,
          n_eval_episodes=n_eval_episodes,
          eval_freq=eval_freq,
          deterministic=deterministic,
          verbose=verbose
      )
      self.trial = trial
      self.eval_idx = 0
      self.is_pruned = False

  def _on_step(self) -> bool:
    if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
      super()._on_step()
      self.eval_idx += 1
      self.trial.report(self.last_mean_reward, self.eval_idx)
      if self.trial.should_prune():
        self.is_pruned = True
        return False
    return True

In [ ]:
def objective(trial: optuna.Trial) -> float:

  kwargs = DEFAULT_HYPERPARAMS.copy()

  sampled_params = sample_a2c_params(trial)
  kwargs.update(sampled_params)

  model = A2C(**kwargs)

  eval_envs = make_vec_env(ENV_ID, n_envs=N_EVAL_ENVS)

  eval_callback = TrialEvalCallback(
    eval_env=eval_envs,
    trial=trial,
    n_eval_episodes=N_EVAL_EPISODES,
    eval_freq=EVAL_FREQ,
    deterministic=True,
    verbose=1
  )

  nan_encountered = False
  try:
    model.learn(N_TIMESTEPS, callback=eval_callback)
  except AssertionError as e:
    print(e)
    nan_encountered = True
  finally:
    model.env.close()
    eval_envs.close()

  if nan_encountered:
    return float("nan")
  if eval_callback.is_pruned:
    raise optuna.exceptions.TrialPruned()

  return eval_callback.last_mean_reward

In [ ]:
th.set_num_threads(1)
sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)
pruner = MedianPruner(
    n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS // 3
)
study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")

try:
    study.optimize(objective, n_trials=N_TRIALS, n_jobs=N_JOBS, timeout=TIMEOUT)
except KeyboardInterrupt:
    pass

print(f"Number of finished trials: {len(study.trials)}")

print("Best trial:")
trial = study.best_trial

print(f"  Value: {trial.value}")

print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

print("  User attrs:")
for key, value in trial.user_attrs.items():
    print(f"    {key}: {value}")

study.trials_dataframe().to_csv("study_results_a2c_cartpole.csv")

fig1 = plot_optimization_history(study)
fig2 = plot_param_importances(study)

fig1.show()
fig2.show()

[I 2024-09-07 08:23:04,051] A new study created in memory with name: no-name-d405db12-7247-44f0-9c64-24d886fd0722
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


Eval num_timesteps=10000, episode_reward=9.30 +/- 0.64
Episode length: 9.30 +/- 0.64
New best mean reward!


[I 2024-09-07 08:23:40,393] Trial 0 finished with value: 9.7 and parameters: {'gamma': 0.013270456736615008, 'max_grad_norm': 1.876085544210284, 'exponent_n_steps': 7, 'learning_rate': 0.06457704378453519, 'net_arch': 'small', 'activation_fn': 'tanh'}. Best is trial 0 with value: 9.7.


Eval num_timesteps=20000, episode_reward=9.70 +/- 0.46
Episode length: 9.70 +/- 0.46
New best mean reward!


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


...

Eval num_timesteps=10000, episode_reward=242.70 +/- 13.40
Episode length: 242.70 +/- 13.40
New best mean reward!


[I 2024-09-07 08:37:42,434] Trial 35 pruned. 


Eval num_timesteps=10000, episode_reward=71.80 +/- 26.64
Episode length: 71.80 +/- 26.64
New best mean reward!
Eval num_timesteps=10000, episode_reward=500.00 +/- 0.00
Episode length: 500.00 +/- 0.00
New best mean reward!
Eval num_timesteps=20000, episode_reward=428.20 +/- 79.45
Episode length: 428.20 +/- 79.45


[I 2024-09-07 08:38:14,556] Trial 36 finished with value: 428.2 and parameters: {'gamma': 0.009145735594131654, 'max_grad_norm': 1.4340150367351903, 'exponent_n_steps': 9, 'learning_rate': 0.0023642271037146146, 'net_arch': 'tiny', 'activation_fn': 'tanh'}. Best is trial 3 with value: 500.0.


Number of finished trials: 37
Best trial:
  Value: 500.0
  Params: 
    gamma: 0.0077638455987499
    max_grad_norm: 4.903937907385273
    exponent_n_steps: 7
    learning_rate: 0.004403423670865376
    net_arch: small
    activation_fn: tanh
  User attrs:
    gamma: 0.9922361544012501
    n_steps: 128
